In [50]:
import pandas as pd
import numpy as np
import seaborn as sns

import src.read_data as read

In [ ]:
def define_veh_string(dtw_data,trajs_data, max_dtw):
    #allows for the cut of cut-in cenarios
    leaders = np.array(dtw_data['leader'])
    followers = np.array(dtw_data['id'])
    dtw_value = np.array(dtw_data['dtw'])
    string_list = []
    string = []
    for k in range(1,len(leaders)):
        if leaders[k]==followers[k-1] and dtw_value[k] < max_dtw:
            string.append(followers[k-1])
        else : 
            string_list.append(string)
            string = []
    return string_list


def define_CF(dtw_data,max_dtw):
    dtw_data['isCF'] = dtw_data['DTW'].apply(lambda x: True if x < max_dtw else False)
    return dtw_data

def determine_CF(trajsdf,DTW_df,max_dtw):
    DTW_df = define_CF(DTW_df,max_dtw)
    out = pd.DataFrame(columns = trajsdf.columns)
    out['CF'] = []
    for k in pd.unique(trajsdf['ID']):
        CF = []
        data = trajsdf[trajsdf['ID']==k]
        dtw_data = DTW_df[DTW_df['leader']==k]
        data.reset_index(inplace=True,drop=True)
        dtw_data.reset_index(inplace=True,drop=True)
        try:
            time_index = 0
            time = dtw_data['time'][time_index+1]
            for t in data['time']:
                if t<time:
                    CF.append(dtw_data['isCF'][time_index])
                elif t>time and time_index<len(dtw_data['time']):
                    if time_index + 1 < len(dtw_data['time']):
                        time_index+=1
                        if time_index + 1 < len(dtw_data['time']):
                            time = dtw_data['time'][time_index+1]
                    CF.append(dtw_data['isCF'][time_index])
                elif t>time and time_index==len(dtw_data['time']):
                    CF.append(dtw_data['isCF'][time_index])
        except Exception:
            pass
        if len(CF)<len(data['time']):
            CF+=[np.nan for k in range(len(data['time'])-len(CF))]
        data['CF']=CF
        out = pd.concat([out,data])
    return out

In [ ]:
DTW_1 = pd.read_csv('out/DTW/DTW_1.csv')
DTW_1 = define_CF(DTW_1,1.5)
DTW_1['time init'] = DTW_1['time']-5
DTW_1['time final'] = DTW_1['time']+25
        

In [ ]:
TRAJS = pd.read_csv('data/by_run/corrected/1.csv')

In [ ]:
DTW_1 = pd.merge(DTW_1, TRAJS[['leader', 'time', 'lane-kf','r']], on=['leader', 'time'], how='inner')
DTW_1

In [ ]:
DTW_1_lane4 = DTW_1[DTW_1['lane-kf']==4]
DTW_1_lane4.sort_values(by=['time','r'], ascending=[True, False],inplace = True)
DTW_1_lane4.reset_index(inplace=True)
display(DTW_1_lane4)
string_list = [True if DTW_1_lane4['isCF'][0]==True else False]
for k in range(1,len(DTW_1_lane4['id'])):
    if DTW_1_lane4['leader'][k]==DTW_1_lane4['id'][k-1] and DTW_1_lane4['isCF'][k]==True:
        string_list.append(True)
    else:
        string_list.append(False)
DTW_1_lane4['string'] = string_list

In [ ]:
DTW_1_lane4

In [ ]:
strings = [np.nan if DTW_1_lane4['string'][0]==False else 0]
string_number = 0
for k in range(1,len(DTW_1_lane4['string'])):
    if DTW_1_lane4['string'][k]==True and DTW_1_lane4['string'][k-1]==False:
        string_number+=1
        strings.append(string_number) 
    elif DTW_1_lane4['string'][k]==True and DTW_1_lane4['string'][k-1]==True:
        strings.append(string_number) 
    else : 
        strings.append(np.nan)
DTW_1_lane4['string number'] = strings
DTW_1_lane4

In [ ]:
DTW_1_lane4[DTW_1_lane4['string number']==2]

In [ ]:
import fastdtw as fastdtw

In [ ]:
min(TRAJS[(TRAJS['ID']==9)]['time'])

In [ ]:
xlead = TRAJS[(TRAJS['ID']==9)&(TRAJS['time']>5)&(TRAJS['time']<35)]
xfollow = TRAJS[(TRAJS['ID']==8)&(TRAJS['time']>6.2)&(TRAJS['time']<36.2)]
X1 = differencing(np.array(xlead['r']), smooth=0.1)
X2 = differencing(np.array(xfollow['r']), smooth=0.1)
distance, paths = dtw.warping_paths(X1, X2)

In [ ]:
distance

In [ ]:
DTW, time =  test_car_following_by_time(TRAJS,9,8,1.2,30,5)

In [ ]:
sns.scatterplot(data = TRAJS[(TRAJS['ID']==9)&(TRAJS['time']>5)&(TRAJS['time']<35)], x = 'time',y = 'r', label = '9')
sns.scatterplot(data = TRAJS[(TRAJS['ID']==8)&(TRAJS['time']>6.2)&(TRAJS['time']<36.2)], x = 'time',y = 'r')

In [ ]:
DTW_1_lane4[DTW_1_lane4['string number']==6]


In [ ]:
from dtaidistance import dtw
from dtaidistance.preprocessing import differencing
def test_platoon(DTW_data,trajs,threshold, window, tau_list):
    
    time_list = []
    out = pd.DataFrame()
    DTW_data = define_CF(DTW_data,threshold)
    DTW_data = pd.merge(DTW_data, trajs[['leader', 'time', 'lane-kf','r']], on=['leader', 'time'], how='inner')
    for k in pd.unique(DTW_data['lane-kf']):
        DTW_data_lane = DTW_data[DTW_data['lane-kf']==k]
        DTW_data_lane.sort_values(by=['time','r'], ascending=[True, False],inplace = True)
        DTW_data_lane.reset_index(inplace=True)
        string_list = [True if DTW_data_lane['isCF'][0]==True else False]
        for i in range(1,len(DTW_data_lane['id'])):
            if DTW_data_lane['leader'][i]==DTW_data_lane['id'][i-1] and DTW_data_lane['isCF'][i]==True:
                string_list.append(True)
            else:
                string_list.append(False)
        DTW_data_lane['string'] = string_list
        strings = [np.nan if DTW_1_lane4['string'][0]==False else 0]
        string_number = 0
        for k in range(1,len(DTW_1_lane4['string'])-1):
            print(k)
            if DTW_data_lane['string'][k]==True and DTW_data_lane['string'][k-1]==False:
                string_number+=1
                strings.append(string_number) 
            elif DTW_data_lane['string'][k]==True and DTW_data_lane['string'][k-1]==True:
                strings.append(string_number) 
            else : 
                strings.append(np.nan)
        DTW_data_lane['string number'] = strings
        out = pd.concat([out,DTW_data_lane])
    return out




In [57]:
from dtaidistance import dtw
from dtaidistance.preprocessing import differencing


def define_strings(DTW_data,TRAJS,dtw_treshold):
    DTW_data  =define_CF(DTW_data,dtw_treshold)
    DTW_data = pd.merge(DTW_data, TRAJS[['leader', 'time', 'lane-kf','r']], on=['leader', 'time'], how='inner')
    temp = pd.DataFrame()
    for k in pd.unique(DTW_data['lane-kf']):
            DTW_data_lane = DTW_data[DTW_data['lane-kf']==k]
            DTW_data_lane.sort_values(by=['time','r'], ascending=[True, False],inplace = True)
            DTW_data_lane.reset_index(inplace=True)
            string_list = [True if DTW_data_lane['isCF'][0]==True else False]
            for i in range(1,len(DTW_data_lane['id'])):
                if DTW_data_lane['leader'][i]==DTW_data_lane['id'][i-1] and DTW_data_lane['isCF'][i]==True:
                    string_list.append(True)
                else:
                    string_list.append(False)
            DTW_data_lane['string'] = string_list
            temp = pd.concat([temp,DTW_data_lane])

    DTW_data = pd.merge(DTW_data, temp[['leader', 'time','string']], on=['leader', 'time'], how='inner')

    DTW_data.sort_values(by=['lane-kf','time','r'], ascending=[True,True, False],inplace = True)
    DTW_data.reset_index(inplace = True,drop = True)
    
    strings = [np.nan if DTW_data['string'][0]==False else 0]
    string_number = 0
    for k in range(1,len(DTW_data['string'])):
                if DTW_data['string'][k]==True and DTW_data['string'][k-1]==False:
                    string_number+=1
                    strings.append(string_number) 
                elif DTW_data['string'][k]==True and DTW_data['string'][k-1]==True:
                    strings.append(string_number) 
                else : 
                    strings.append(np.nan)
    DTW_data['string number'] = strings
    return DTW_data

def compute_dtw_total_platoon(DTW_string_df,TRAJS,window, tau_list):
    DTW_string_df = DTW_string_df.dropna(subset = 'string number')
    DTW_list = []
    leader_list = []
    id_list = []
    time = []
    for k in pd.unique(DTW_string_df['string number']):
        string_df = DTW_string_df[DTW_string_df['string number']==k]
        tmin = min(string_df['time'])
        tmax = tmin + window
        ids_string = pd.unique(string_df['id'])
        try : 
            Xlead = differencing(np.array(TRAJS[(TRAJS['time']>tmin)&(TRAJS['time']<tmax)&(TRAJS['ID']==string_df.reset_index()['leader'][0])]['r']), smooth=0.1)   
            for k in range(len(ids_string)):
                try :
                    distance_list = []
                    for tau in tau_list :
                        Xfollow = differencing(np.array(TRAJS[(TRAJS['time']>tmin+tau*(k))&(TRAJS['time']<tmax+tau*(k))&(TRAJS['ID']==ids_string[k])]['r']), smooth=0.1)
                        distance, paths = dtw.warping_paths(Xlead, Xfollow)
                        distance_list.append(distance)
                    DTW_list.append(np.array(distance))
                    leader_list.append(string_df.reset_index()['leader'][0])
                    id_list.append(ids_string[k])
                    time.append(tmin)
                except Exception: 
                    DTW_list.append(min(distance_list))
                    leader_list.append(string_df.reset_index()['leader'][0])
                    id_list.append(ids_string[k])
                    time.append(tmin)
        except Exception:
            for k in range (len(ids_string)):
                DTW_list.append(np.nan)
                leader_list.append(string_df.reset_index()['leader'][0])
                id_list.append(ids_string[k])
                time.append(tmin)
    string_evaluation  = pd.DataFrame({"id" : id_list, 'leader_string' : leader_list,'DTW string' : DTW_list, 'time':time})
    string_evaluation['string_number'] = DTW_string_df['string number']
    return string_evaluation

In [58]:
path_trajs = 'data/by_run/corrected/'
path_dtw = 'out/DTW/'
trajs_list = ['1.csv', '11.csv', '18.csv', '19.csv', '20.csv', '21.csv', '3.csv', '7.csv', '8.csv', '9.csv']
dtw_list = ['DTW_1.csv', 'DTW_11.csv', 'DTW_18.csv', 'DTW_19.csv', 'DTW_20.csv', 'DTW_21.csv', 'DTW_3.csv', 'DTW_7.csv', 'DTW_8.csv', 'DTW_9.csv']
for k in range(len(trajs_list)):
    DTW = pd.read_csv(path_dtw + str(dtw_list[k]))
    TRAJS = pd.read_csv(path_trajs + str(trajs_list[k]))
    DTW = define_strings(DTW,TRAJS,1.5)
    string_dtw = compute_dtw_total_platoon(DTW,TRAJS,30, 1.2)
    string_dtw.to_csv('out/string_dtw/'+str(trajs_list[k]))


NameError: name 'define_CF' is not defined

In [ ]:
DTW_1_strings = pd.read_csv('out/string_dtw/1.csv')
trajs = pd.read_csv( 'data/by_run/corrected/1.csv')
ACC = pd.unique(trajs[trajs['ACC']=='Yes']['ID'])
out = pd.DataFrame()
for k in pd.unique(DTW_1_strings['leader_string']):
    data = DTW_1_strings[DTW_1_strings['leader_string']==k]
    data['string'] = [k for i in range(len(data['id']))]
    out = pd.concat([out,data])
out['ACC'] = out['id'].isin(ACC)

In [ ]:
trajs_list = ['1.csv', '11.csv', '18.csv', '19.csv', '20.csv', '21.csv', '3.csv', '7.csv', '8.csv', '9.csv']
for k in range(len(trajs_list)):
    DTW_1_strings = pd.read_csv('out/string_dtw/'+trajs_list[k])
    trajs = pd.read_csv( 'data/by_run/corrected/'+trajs_list[k])
    ACC = pd.unique(trajs[trajs['ACC']=='Yes']['ID'])
    out = pd.DataFrame()
    for s in pd.unique(DTW_1_strings['leader_string']):
        data = DTW_1_strings[DTW_1_strings['leader_string']==s]
        data['string'] = [k for i in range(len(data['id']))]
        out = pd.concat([out,data])
    out['ACC'] = out['id'].isin(ACC)
    results = study_platoon_evolution(out)
    results.to_csv('out/platoons/'+trajs_list[k])

In [ ]:
trajs_8 = pd.read_csv('data/by_run/corrected/8.csv')
trajs8ACC = pd.unique(trajs_8[trajs_8['ACC']=='Yes']['ID'])

In [ ]:
trajs8ACC

In [ ]:
platoon_8 = pd.read_csv('out/platoons/8.csv')
platoon_8

In [ ]:
out[out['leader_string']==105]

In [ ]:
out[out['ACC']]

In [ ]:
max(trajs[trajs['ACC']=='Yes']['time'])

In [ ]:
def study_platoon_evolution(out):
    results = pd.DataFrame()
    for k in pd.unique(out['leader_string']):
        test = out[out['leader_string']==k]
        begin = min(test['time'])
        init = test[test['time'] == pd.unique(test['time'])[0]]
        if (init['DTW string'] ).all():
            duration = 30
            compo = pd.unique(init['id'])
        else :
            compo = []
        for t in pd.unique(test['time']):
            init = test[test['time'] == t]
            if len(compo)>0:
                if (init['DTW string'].values ).all()< 1.5 and len(pd.unique(init['id']))==len(compo):
                    duration += 5
                elif (init['DTW string'] ).all()< 1.5 and len(pd.unique(init['id']))<len(compo): 
                    results = pd.concat([results, pd.DataFrame({'leader' : k, 'compo': [compo], 'begin' : begin, 'duration': duration, 'reason' : 'extraction'})])
                    compo = pd.unique(init['id'])
                    begin = t
                    duration = 30
                elif  (init['DTW string'] < 1.5).all() and len(pd.unique(init['id']))>len(compo): 
                    results = pd.concat([results, pd.DataFrame({'leader' : k, 'compo': [compo], 'begin' : begin, 'duration': duration, 'reason' : 'insertion'})])
                    compo = pd.unique(init['id'])
                    begin = t
                    duration = 30
                else :
                    results = pd.concat([results, pd.DataFrame({'leader' : k, 'compo': [compo], 'begin' : begin, 'duration': duration, 'reason' : 'disengagement'})])
                    compo = []
            else : 
                if (init['DTW string'].values ).all()< 1.5:
                    compo = pd.unique(init['id'])
                    duration = 30
                    begin = t
    return results

In [ ]:
results = study_platoon_evolution(out)
results

In [ ]:
results = pd.DataFrame()
for k in pd.unique(out['leader_string']):
    test = out[out['leader_string']==k]
    test = test[test['DTW string']<1.5].reset_index()
    begin = min(test['datetime'])
    increase = np.nan
    try : 
        compo = pd.unique(test[test['time'] == pd.unique(test['time'])[0]]['id'])
        duration = 30
        lenght = len(compo)
        for t in pd.unique(test['time']):
            trial = test[test['time']==t]
            if np.array_equal(compo,pd.unique(trial['id'])):
                duration+=5
            elif np.array_equal(compo,pd.unique(trial['id']))==False and len(compo)<len(pd.unique(trial['id'])):
                duration+=5
                increase = duration
                lenght+=1
            else : 
                break
        if lenght>1:
            lenght +=1
            print(compo)
            results = pd.concat([results,pd.DataFrame({'leader':[k for i in range(len(compo))],'followers':list(compo),'duration':duration,'lenght':lenght,
                                                       'increase time':increase})])
    except Exception:
        pass
results

In [ ]:
out[out['ACC']==True]

In [ ]:
DTW_1_strings = define_strings(DTW_1,TRAJS,1.5)

In [ ]:

window = 30
tau = 1.2


In [ ]:
len(leader_list)

In [ ]:
string_evaluation  = pd.DataFrame({"id" : id_list, 'leader_string' : leader_list,'DTW string' : DTW_list, 'time':time})
string_evaluation['string_number'] = DTW_1_strings['string number']
string_evaluation

In [ ]:
DTW_1_strings

In [ ]:
DTW_string_values = pd.DataFrame({"id" : id_list, 'leader_string' : leader_list,'DTW string' : DTW_list, 'time':time}).merge(DTW_1_strings,on = ['id','time'], how = 'inner')
DTW_string_values

In [ ]:
DTW_1_strings['string number']

In [ ]:
DTW_1_strings.reset_index(inplace = True)

In [ ]:
DTW_1_strings['DTW string'] = DTW_list

In [ ]:
DTW_1

In [ ]:
DTW_1 = pd.read_csv('out/DTW/DTW_1.csv')
DTW_1_string_eval = test_platoon(DTW_1,TRAJS,1.5, 30, tau = 1.2, fixed_tau =True)


In [ ]:


        for s in pd.unique(DTW_data_lane['string number']):
            DTW_list = []
            DTW_string = DTW_data_lane[DTW_data_lane['string number']==s].reset_index(drop=True)
            try : 
                tmin = min(DTW_string['time'])
                tmax = tmin + window
                Xlead = differencing(np.array(TRAJS[(TRAJS['time']>tmin)&(TRAJS['time']<tmax)&(TRAJS['ID']==DTW_string.reset_index()['leader'][0])]['r']), smooth=0.1)
                ids_string = pd.unique(DTW_string['id'])
                for k in range(len(ids_string)):
                    Xfollow = differencing(np.array(TRAJS[(TRAJS['time']>tmin+tau*(k))&(TRAJS['time']<tmax+tau*(k))&(TRAJS['ID']==ids_string[k])]['r']), smooth=0.1)
                    distance, paths = dtw.warping_paths(Xlead, Xfollow)
                    DTW_list.append(distance)
            except Exception:
                    distance = np.nan
                    DTW_list.append(distance)
            DTW_string['string DTW'] = DTW_list
            out = pd.concat([out, DTW_string])

    return out

In [ ]:
test = DTW_1_lane4[DTW_1_lane4['string number']==6].reset_index()
test

In [ ]:
window = 30
tau = 1.2

In [ ]:
tmin = test['time'][0]
tmax = tmin+window
DTW_list = []
Xlead = differencing(np.array(TRAJS[(TRAJS['time']>tmin)&(TRAJS['time']<tmax)&(TRAJS['ID']==test['leader'][0])]['r']), smooth=0.1)
followers = pd.unique(test['id'])
for k in range(len(followers)):
    Xfollow = np.array(TRAJS[(TRAJS['time']>tmin+tau*(k+1))&(TRAJS['time']<tmax+tau*(k+1))&(TRAJS['ID']==followers[k])]['r'])
    Xfollow = differencing(Xfollow, smooth=0.1)
    d, p = distance, paths = dtw.warping_paths(Xlead, Xfollow)
    DTW_list.append(d)
DTW_list

In [ ]:
Xlead = differencing(np.array(TRAJS[(TRAJS['time']>10)&(TRAJS['time']<40)&(TRAJS['ID']==DTW_1_lane4[DTW_1_lane4['string number']==2].reset_index()['leader'][0])]['r']), smooth=0.1)
Xfollow = differencing(np.array(TRAJS[(TRAJS['time']>11.2)&(TRAJS['time']<41.2)&(TRAJS['ID']==DTW_1_lane4[DTW_1_lane4['string number']==2].reset_index()['id'][0])]['r']), smooth=0.1)
Xfollow2 = differencing(np.array(TRAJS[(TRAJS['time']>12.4)&(TRAJS['time']<42.4)&(TRAJS['ID']==DTW_1_lane4[DTW_1_lane4['string number']==2].reset_index()['id'][1])]['r']), smooth=0.1)
Xfollow3 = differencing(np.array(TRAJS[(TRAJS['time']>12.4)&(TRAJS['time']<42.4)&(TRAJS['ID']==DTW_1_lane4[DTW_1_lane4['string number']==2].reset_index()['id'][2])]['r']), smooth=0.1)


In [ ]:
distance, paths = dtw.warping_paths(Xlead, Xfollow)
distance2, paths2 = dtw.warping_paths(Xlead, Xfollow2)
distance3, paths3 = dtw.warping_paths(Xlead, Xfollow3)
distance4, paths4 = dtw.warping_paths(Xlead, Xfollow4)


In [ ]:
print(distance,distance2,distance3,distance4)

In [ ]:
DTW_1_lane4

In [ ]:
from dtaidistance import dtw
from dtaidistance.preprocessing import differencing
import matplotlib.pyplot as plt
#assuming a constant translation of 1.2s is valid
tau = 1.2
DTW_list =  []
plt.plot(TRAJS[(TRAJS['time']>5)&(TRAJS['time']<35)&(TRAJS['ID']==DTW_1_lane4[DTW_1_lane4['string number']==2].reset_index()['id'][0])]['time'],TRAJS[(TRAJS['time']>5)&(TRAJS['time']<35)&(TRAJS['ID']==DTW_1_lane4[DTW_1_lane4['string number']==2].reset_index()['id'][0])]['r'])
Xlead = differencing(np.array(TRAJS[(TRAJS['time']>5)&(TRAJS['time']<35)&(TRAJS['ID']==DTW_1_lane4[DTW_1_lane4['string number']==2].reset_index()['id'][0])]['r']), smooth=0.1)
ids_string = pd.unique(DTW_1_lane4[DTW_1_lane4['string number']==2]['id'])
for k in range(1,len(ids_string)):
    Xfollow = differencing(np.array(TRAJS[(TRAJS['time']>5+tau*(k))&(TRAJS['time']<35+tau*(k))&(TRAJS['ID']==ids_string[k])]['r']), smooth=0.1)
    distance, paths = dtw.warping_paths(Xlead, Xfollow)
    DTW_list.append(distance)
    plt.plot(TRAJS[(TRAJS['time']>5+tau*(k))&(TRAJS['time']<35+tau*(k))&(TRAJS['ID']==ids_string[k])]['time'],TRAJS[(TRAJS['time']>5+tau*(k+1))&(TRAJS['time']<35+tau*(k+1))&(TRAJS['ID']==ids_string[k])]['r'])

In [ ]:
DTW_list

In [ ]:
DTW_list

In [ ]:
DTW_1.sort_values(by = ['time','id','leader'])

In [ ]:
for k in DTW_1

In [ ]:
def stest_string_over_time(data,threshold):
    for k in pd.unique(trajsdf['ID']):
        

In [ ]:
DTW_1['time init'] = DTW_1['time']-5
DTW_1['time final'] = DTW_1['time']+25
DTW_1[DTW_1['id']==12]
DTW_1[DTW_1['leader']==12]